In [7]:
import pandas as pd

# --- 설정 ---
# 원본 파일 (필터링을 위해 'magnitude' 컬럼이 있는 원본 파일)
input_file = 'earthquakes_1.csv'
# 최종 결과 파일
output_file = 'earthquakes_filtered_include_depth.csv'
# ---------------------

print(f"'{input_file}' 파일을 불러와 처리를 시작합니다.")

try:
    # 1. 원본 CSV 파일 불러오기
    df = pd.read_csv(input_file)

    # 2. 필터링에 필요한 컬럼이 있는지 확인
    required_cols = ['date', 'tsunami', 'magnitude', 'latitude', 'longitude']
    if not all(col in df.columns for col in required_cols):
        missing = [col for col in required_cols if col not in df.columns]
        print(f"오류: 필수 컬럼이 누락되었습니다: {missing}")
        exit()

    # 3. 'magnitude' 컬럼을 숫자로 변환 (오류 발생 시 NaN으로)
    df['magnitude'] = pd.to_numeric(df['magnitude'], errors='coerce')

    # 4. [수정된 필터링 로직]

    # 조건 1: tsunami가 1인 데이터 (모두 유지)
    condition1 = (df['tsunami'] == 1)

    # 조건 2: tsunami가 0이면서, magnitude가 5.0 초과인 데이터
    condition2 = (df['tsunami'] == 0) & (df['magnitude'] > 5.0)

    # 조건 1(tsunami=1) 또는 조건 2(tsunami=0 AND mag>5.0)를 만족하는 행 선택
    # .copy()를 사용하여 SettingWithCopyWarning 방지
    df_filtered = df[condition1 | condition2].copy()

    print("필터링 적용 완료:")
    print("1. 'tsunami == 1'인 모든 데이터 유지")
    print("2. 'tsunami == 0' 이면서 'magnitude > 5.0'인 데이터 유지")
    print(f"필터링 후 남은 데이터 수: {len(df_filtered)}")

    # 5. [기존 요청] 날짜 처리 (필터링된 데이터 대상)
    df_filtered['date'] = pd.to_datetime(df_filtered['date'], errors='coerce')
    df_filtered['Year'] = df_filtered['date'].dt.year
    df_filtered['Month'] = df_filtered['date'].dt.month

    # 6. [기존 요청] 최종적으로 유지할 컬럼 선택
    final_columns_to_keep = ['tsunami', 'magnitude','depth','latitude', 'longitude', 'Year', 'Month']

    # 7. 최종 DataFrame 생성 및 날짜 변환 실패(NaT/NaN) 데이터 제거
    df_final = df_filtered[final_columns_to_keep].dropna()

    # 8. (선택 사항) Year와 Month를 정수형(int)으로 변환
    if 'Year' in df_final.columns:
        df_final['Year'] = df_final['Year'].astype(int)
    if 'Month' in df_final.columns:
        df_final['Month'] = df_final['Month'].astype(int)

    # 9. 새 CSV 파일로 저장
    df_final.to_csv(output_file, index=False)

    print(f"\n처리가 완료되었습니다. '{output_file}'로 저장되었습니다.")
    print("\n최종 데이터 상위 5개 행:")
    print(df_final.head())

    print("\n최종 데이터의 tsunami 값 분포 (1이 포함되어 있는지 확인):")
    print(df_final['tsunami'].value_counts())

except FileNotFoundError:
    print(f"오류: '{input_file}' 파일을 찾을 수 없습니다.")
    print(f"스크립트와 같은 폴더에 '{input_file}' 파일이 있는지 확인하세요.")
except Exception as e:
    print(f"처리 중 예외가 발생했습니다: {e}")

'earthquakes_1.csv' 파일을 불러와 처리를 시작합니다.
필터링 적용 완료:
1. 'tsunami == 1'인 모든 데이터 유지
2. 'tsunami == 0' 이면서 'magnitude > 5.0'인 데이터 유지
필터링 후 남은 데이터 수: 661

처리가 완료되었습니다. 'earthquakes_filtered_include_depth.csv'로 저장되었습니다.

최종 데이터 상위 5개 행:
    tsunami  magnitude    depth  latitude  longitude  Year  Month
1         0       5.10   6.1279   32.4140   -102.057  2024      9
8         1       4.69  11.3500   34.0703   -118.808  2024      9
15        1       3.94   4.8400   34.0297   -117.586  2024      9
18        1       4.28   0.1600   38.7765   -122.746  2024      9
51        1       4.39  10.9800   34.0843   -118.185  2024      8

최종 데이터의 tsunami 값 분포 (1이 포함되어 있는지 확인):
tsunami
0    593
1     68
Name: count, dtype: int64
